In [1]:
!nvidia-smi

Thu Jun  2 10:31:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -q transformers
!pip install -q datasets
!pip install -q sentence-transformers

In [4]:
import os, sys
import random
from pathlib import Path
from tqdm.notebook import tqdm
from typing import Dict


import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from datasets import load_metric

from transformers import RobertaForMaskedLM, RobertaTokenizerFast, RobertaTokenizer
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput

from sentence_transformers import InputExample
from sentence_transformers import models, SentenceTransformer
from sentence_transformers import losses
from sentence_transformers.util import cos_sim
from sentence_transformers.evaluation import LabelAccuracyEvaluator
from sentence_transformers.losses.TripletLoss import TripletDistanceMetric

# Prepare Dataset

## Load Dataframe dataset

In [5]:
train_df = pd.read_pickle('/content/drive/MyDrive/NLP/AI4Code/dataset/exp_5/clean_train_dataset.pkl')
train_df

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
0,00001756c60be8,1862f0a6,code,# this python 3 environment comes with many he...,0,945aea18,NaN,0.0
1,00001756c60be8,2a9e43d6,code,import numpy as np import pandas as pd import ...,2,945aea18,NaN,0.034483
2,00001756c60be8,038b763d,code,import warnings warnings.filterwarnings('ignore'),4,945aea18,NaN,0.068966
3,00001756c60be8,2eefe0ef,code,matplotlib.rcparams.update({'font.size': 14}),6,945aea18,NaN,0.103448
4,00001756c60be8,0beab1cd,code,"def evaluate_preds(train_true_values, train_pr...",8,945aea18,NaN,0.137931
...,...,...,...,...,...,...,...,...
6055708,fffe1d764579d5,0d770d6b,markdown,removing the outliers,43,3c40bfa6,NaN,0.597222
6055709,fffe1d764579d5,d45ddc62,markdown,dimensionality curse,33,3c40bfa6,NaN,0.458333
6055710,fffe1d764579d5,1a63248d,markdown,bangalore house price prediction,0,3c40bfa6,NaN,0.0
6055711,fffe1d764579d5,a8ffc8b4,markdown,we have achieved accuracy in predicting the pr...,69,3c40bfa6,NaN,0.958333


In [6]:
val_df = pd.read_pickle('/content/drive/MyDrive/NLP/AI4Code/dataset/exp_5/clean_val_dataset.pkl')
val_df

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
0,0002115f48f982,18281c6c,code,import numpy as np # linear algebra import pan...,1,272b483a,NaN,0.111111
1,0002115f48f982,e3b6b115,code,df = pd.read_csv('../input/metadata_train.csv'...,2,272b483a,NaN,0.222222
2,0002115f48f982,4a044c54,code,df.head(),3,272b483a,NaN,0.333333
3,0002115f48f982,365fe576,code,#let's check if targets are consistent within ...,4,272b483a,NaN,0.444444
4,0002115f48f982,a3188e54,code,"sns.countplot(x='target',data=targets) # it sh...",5,272b483a,NaN,0.555556
...,...,...,...,...,...,...,...,...
314928,fff06cc23780af,f8135651,markdown,lowering the case,14,7b4c5375,NaN,0.17284
314929,fff06cc23780af,b61ba8ec,markdown,testing the model on test set,60,7b4c5375,NaN,0.740741
314930,fff06cc23780af,e98b7e0d,markdown,the aim of this notebook is to predict if twee...,2,7b4c5375,NaN,0.024691
314931,fff06cc23780af,f31fa490,markdown,having created this notebook from scratch star...,80,7b4c5375,NaN,0.987654


## Create cellid--source dictionary

In [7]:
train_dict_cellid_source = dict(zip(train_df['cell_id'].values, train_df['source'].values))
val_dict_cellid_source = dict(zip(val_df['cell_id'].values, val_df['source'].values))

In [8]:
def generate_triplet(df):
    triplets = []
    count = 0

    for id, df_tmp in tqdm(df.groupby('id')):
        df_tmp_markdown = df_tmp[df_tmp['cell_type'] == 'markdown']
        df_tmp_code = df_tmp[df_tmp['cell_type'] == 'code']

        df_tmp_code_rank = df_tmp_code['rank'].values
        df_tmp_code_cellid = df_tmp_code['cell_id'].values

        for cell_id, rank in df_tmp_markdown[['cell_id', 'rank']].values:
            labels = np.array([r == (rank+1) for r in df_tmp_code_rank]).astype('int')

            pos_position = np.where(labels == 1)[0]

            if len(pos_position):
                pos_position = pos_position[0]
                pos_cellid = df_tmp_code_cellid[pos_position]
                pos_label = labels[pos_position]
                triplets.append([cell_id, pos_cellid])

    return triplets

In [9]:
def generate_triplet_val(df):
    triplets = []
    count = 0

    for id, df_tmp in tqdm(df.groupby('id')):
        df_tmp_markdown = df_tmp[df_tmp['cell_type'] == 'markdown']
        df_tmp_code = df_tmp[df_tmp['cell_type'] == 'code']

        df_tmp_code_rank = df_tmp_code['rank'].values
        df_tmp_code_cellid = df_tmp_code['cell_id'].values

        for cell_id, rank in df_tmp_markdown[['cell_id', 'rank']].values:
            labels = np.array([r == (rank+1) for r in df_tmp_code_rank]).astype('int')

            pos_position = np.where(labels == 1)[0]

            if len(pos_position):
                pos_position = pos_position[0]
                pos_cellid = df_tmp_code_cellid[pos_position]
                pos_label = labels[pos_position]

                neg_positions = [x for x in range(0, len(labels)) if x != pos_position]

                if len(neg_positions):

                    neg_positions = random.choices(neg_positions, k=5)
                    neg_cellids = df_tmp_code_cellid[neg_positions]
                    for neg_cellid in neg_cellids:
                        triplets.append([cell_id, pos_cellid, neg_cellid])
    
    return triplets

## Take fraction of dataset for test running

In [10]:
# train_df = train_df.loc[:600000]
# train_df

In [11]:
# val_df = val_df.loc[:30000]
# val_df

In [12]:
train_triplets = generate_triplet(train_df)
val_triplets = generate_triplet_val(val_df)

  0%|          | 0/132361 [00:00<?, ?it/s]

  0%|          | 0/6895 [00:00<?, ?it/s]

In [13]:
train_triplets[:20]

[['21616367', '86497fe1'],
 ['fcb6792d', '5bf9ca51'],
 ['63c26fa2', '62638fba'],
 ['4bb2e30a', 'bd94f005'],
 ['a6357f7e', 'ff7c44ed'],
 ['45082c89', '781bbf3c'],
 ['77e56113', '2eefe0ef'],
 ['448eb224', '2a9e43d6'],
 ['032e2820', 'a98c5d9f'],
 ['8554b284', '59959af5'],
 ['ac301a84', '0e7c906e'],
 ['23705731', 'ebe125d5'],
 ['1496beaf', '8ca8392c'],
 ['2e1a5949', '80151ab7'],
 ['7e2f170a', '038b763d'],
 ['bfbde93e', '8522781a'],
 ['915643b3', 'f5504853'],
 ['8ffe0b25', '9a78ab76'],
 ['b69a4f9b', '17ec3fc4'],
 ['c3ce0945', 'e2c8e725']]

## Define custom Dataset for Huggingface Trainer

In [14]:
train_samples = []
for triplet in train_triplets:
    markdown_text = train_dict_cellid_source[triplet[0]]
    pos_code_text = train_dict_cellid_source[triplet[1]]
    train_samples.append(InputExample(
        texts=[markdown_text, pos_code_text]))

In [15]:
val_samples = []
for triplet in val_triplets:
    markdown_text = val_dict_cellid_source[triplet[0]]
    pos_code_text = val_dict_cellid_source[triplet[1]]
    neg_code_text = val_dict_cellid_source[triplet[2]]
    val_samples.append(InputExample(
        texts=[markdown_text, pos_code_text, neg_code_text]))

## Training dataloader

In [16]:
BATCH_SIZE = 16

train_dataloader = DataLoader(train_samples, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=0, drop_last=False)

val_dataloader = DataLoader(val_samples, batch_size=BATCH_SIZE, shuffle=False,
                              num_workers=0, drop_last=False)

# Sentence Transformer Model

In [17]:
bert = models.Transformer("/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-160000", max_seq_length=256)
pooler = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
)

model = SentenceTransformer(modules=[bert, pooler])
# model = SentenceTransformer('/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_task/trial_2/')
model

Some weights of the model checkpoint at /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-160000 were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-160000 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Freeze some layer for faster training

In [18]:
# for name, module in model.named_modules():
#     print(name)

In [19]:
# model[0].auto_model.encoder

In [20]:
# for param in model[0].auto_model.encoder.parameters():
#     param.requires_grad = False

# Loss function

In [21]:
loss = losses.MultipleNegativesRankingLoss(model=model)

# Train Model

In [22]:
from sentence_transformers.evaluation import TripletEvaluator

In [23]:
evaluator = TripletEvaluator.from_input_examples(val_samples)

In [24]:
epochs = 4
warmup_steps = int(len(train_dataloader) * epochs * 0.1)
warmup_steps = 0

model.fit(
    train_objectives=[(train_dataloader, loss)],
    epochs=epochs,
    evaluator=evaluator,
    evaluation_steps=22500,
    warmup_steps=warmup_steps,
    output_path='/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_task/trial_5',
    checkpoint_path='/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_task/trial_5',
    optimizer_params={'lr': 2e-5},
    checkpoint_save_steps=30000,
    checkpoint_save_total_limit=2,
    use_amp=True,
    show_progress_bar=True,
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/93862 [00:00<?, ?it/s]

KeyboardInterrupt: ignored